# tabular-playground-series-jan-2022 Modeling
- データに関する知見だけでなく、データ分析の基礎的な方法をコメントで残す形とする

## 1. Import libralies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import itertools

from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedKFold, learning_curve, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

sns.set(style='white', context='notebook', palette='pastel')

## 2. Load processed data

In [2]:
# Load data
#train = pd.read_csv('../data/processed/train_v1_label.csv', index_col=0)
#test = pd.read_csv('../data/processed/test_v1_label.csv', index_col=0)
#train = pd.read_csv('../data/processed/train_v2_onehot.csv', index_col=0)
#test = pd.read_csv('../data/processed/test_v2_onehot.csv', index_col=0)
#train = pd.read_csv('../data/processed/train_v3_onehot_economy.csv', index_col=0)
#test = pd.read_csv('../data/processed/test_v3_onehot_economy.csv', index_col=0)
#train = pd.read_csv('../data/processed/train_v4_onehot_holiday.csv', index_col=0)
#test = pd.read_csv('../data/processed/test_v4_onehot_holiday.csv', index_col=0)
train = pd.read_csv('../data/processed/train_v5_onehot_economy_holiday.csv', index_col=0)
test = pd.read_csv('../data/processed/test_v5_onehot_economy_holiday.csv', index_col=0)

"""
train['num_sold'] = train['num_sold'].astype(int)
Y_train = train['num_sold']
X_train = train.drop(columns=['num_sold'])
"""
train['num_sold_base'] = train['num_sold_base'].astype(int)
Y_train = train['num_sold_base']
X_train = train.drop(columns=['num_sold', 'num_sold_base'])

raw_test = pd.read_csv("../data/raw/test.csv")
test_id = raw_test['row_id']

In [3]:
X_train.head()

,year,month,day,dayofweek,Christmas,NewYear,Easter,Pentecost,country_0,country_1,country_2,store_0,store_1,product_0,product_1,product_2
0,2015,1,1,3,0,1,0,0,1,0,0,1,0,0,1,0
1,2015,1,1,3,0,1,0,0,1,0,0,1,0,1,0,0
2,2015,1,1,3,0,1,0,0,1,0,0,1,0,0,0,1
3,2015,1,1,3,0,1,0,0,1,0,0,0,1,0,1,0
4,2015,1,1,3,0,1,0,0,1,0,0,0,1,1,0,0


## 3. Training Setting

### 3.1 define setting parameters
Cross validate models

In [4]:
# 再現性確保のためのrandom_state の設定
random_state = 43

# Cross validate model with Kfold stratified cross val
# kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)

### 3.2 define evaluation

In [15]:
def calc_smape_score(true, pred):
    """SMAPEを計算する

    true (np.array) : 実測値
    pred (np.array) : 予測値

    np.array        : smapeの計算結果
    """

    return 100 / len(true) * np.sum(2 * np.abs(pred - true) / (np.abs(pred) + np.abs(true)))


def calc_smape_score_for_lgbm(true, pred):
    """SMAPEを計算する

    true (np.array) : 実測値
    pred (np.array) : 予測値

    np.array        : smapeの計算結果
    """
    score = 100 / len(true) * np.sum(2 * np.abs(pred - true) / (np.abs(pred) + np.abs(true)))

    return 'SMAPE', score, False



In [6]:
scoring_smape = make_scorer(calc_smape_score, greater_is_better=False)

score_funcs = {
    'smape': scoring_smape,
    'R2': make_scorer(r2_score, greater_is_better=True),
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
}

### 3.3 split datasets

In [7]:
#x_train.head()

## 4. Modeling

### 4.1 model setting

In [11]:
# LightGBM
LGBM = LGBMRegressor(random_state=random_state,
                     boosting_type='gbdt',
                     objective='regression',
                     n_estimators=8000,
                     learning_rate=0.05,
                     max_depth=9,
                     num_leaves=15,
                     feature_fraction=0.7,
                     )

### 4.2 Baseline result
- パラメータサーチなし
- cross validationのみ実施

In [16]:
years_list = [2016, 2017, 2018]

scores = []

# make validation data
for year in years_list:
    x_train = X_train[train["year"] < year]
    y_train = Y_train.iloc[:len(x_train)]

    x_val = X_train[train["year"] == year]
    y_val = Y_train.iloc[len(x_train):len(x_train)+len(x_val)]

    x_train = x_train.drop(columns=['year'])
    x_val = x_val.drop(columns=['year'])
    #print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

    #scores = cross_validate(LGBM, x_train, y_train, cv=kfold, scoring=score_funcs)
    LGBM_base = LGBM.fit(x_train, y_train, eval_set=(x_val, y_val), early_stopping_rounds=300, eval_metric=calc_smape_score_for_lgbm)
    pred_y_val = LGBM_base.predict(x_val)
    scores.append(calc_smape_score(y_val, pred_y_val))

print(scores)
np.mean(scores)


[1]	valid_0's l2: 54930.8	valid_0's SMAPE: 50.3711
Training until validation scores don't improve for 300 rounds
[2]	valid_0's l2: 51267.8	valid_0's SMAPE: 48.7955
[3]	valid_0's l2: 47077.8	valid_0's SMAPE: 46.9777
[4]	valid_0's l2: 43234	valid_0's SMAPE: 45.2693
[5]	valid_0's l2: 39771.4	valid_0's SMAPE: 43.6546
[6]	valid_0's l2: 36859	valid_0's SMAPE: 42.138
[7]	valid_0's l2: 34029.9	valid_0's SMAPE: 40.6494
[8]	valid_0's l2: 31433.4	valid_0's SMAPE: 39.2028
[9]	valid_0's l2: 29615.2	valid_0's SMAPE: 38.0479
[10]	valid_0's l2: 27438.6	valid_0's SMAPE: 36.7385
[11]	valid_0's l2: 25898.4	valid_0's SMAPE: 35.7082
[12]	valid_0's l2: 24120.1	valid_0's SMAPE: 34.6167
[13]	valid_0's l2: 22600.9	valid_0's SMAPE: 33.5713
[14]	valid_0's l2: 21065.5	valid_0's SMAPE: 32.4604
[15]	valid_0's l2: 19566.3	valid_0's SMAPE: 31.3767
[16]	valid_0's l2: 18305	valid_0's SMAPE: 30.3227
[17]	valid_0's l2: 17208.9	valid_0's SMAPE: 29.3429
[18]	valid_0's l2: 16296.6	valid_0's SMAPE: 28.5427
[19]	valid_0's l2:

5.866785955433741

In [ ]:
LGBM_base = LGBM.fit(x_train, y_train, x_val, y_val)
pred_y_val = LGBM_base.predict(x_val)
calc_smape_score(y_val, pred_y_val)

In [ ]:
mean_smape = scores['test_smape'].mean() * (-1)
print('SMAPE:', mean_smape)

### 4.3 Parameter Tuning
- cross validation に加え、grid search によるパラメータ探索を実施

In [ ]:
# LightGBM parameters
lgb_param_grid = {
                 #"n_estimators": [500, 1000, 2000],
                 "learning_rate": [0.1, 0.05],
                 'num_leaves' : [15, 31, 63],
                 #'max_depth'  : [9, 15, 21],
                 #'min_gain_to_split' : [0, 0.1, 0.2],
                 #'feature_fraction' : [0.5, 0.7, 1],
                 #'bagging_fraction' : [0.7, 0.9, 1],
                 #'min_sum_hessian_in_leaf' : [1, 2, 4],
                }

In [ ]:
gsLGB = GridSearchCV(LGBM, param_grid=lgb_param_grid, scoring=scoring_smape, cv=2, n_jobs=-1, verbose=1)
gsLGB.fit(x_train, y_train, eval_set=(x_val, y_val))

LGB_best = gsLGB.best_estimator_

# Best Score
print(f"Best Score: {gsLGB.best_score_}")
print(f"Best Param: {gsLGB.best_params_}")

### 4.4 grid search results

In [ ]:
param_list = [f"param_{key}" for key in lgb_param_grid.keys()]
param_comb_list = list(itertools.combinations(param_list, 2))
param_comb_list

In [ ]:
cv_result = pd.DataFrame(gsLGB.cv_results_)

for params in param_comb_list:
    plt.figure()

    cv_result_pivot = cv_result.pivot_table('mean_test_score', params[0], params[1])
    heat_map = sns.heatmap(cv_result_pivot, annot=True)

### 4.5 Plot learning curves
- Learning curves are a good way to see the overfitting effect on the training set and the effect of the training size on the accuracy.

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve
    """
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    if ylim is not None:
        plt.ylim(*ylim)

    print(train_sizes)
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=-1, train_sizes=train_sizes)
    print(train_sizes)
    print(len(X))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    print(train_scores_std)
    print(test_scores_std)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                     alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-varidation score")

    plt.legend(loc="best")
    return plt

#g = plot_learning_curve(GBC_best, "GradientBoosting learning curves", X_train, Y_train, cv=kfold)
#g = plot_learning_curve(RFC_best, "RF learning curves", X_train, Y_train, cv=kfold)
#g = plot_learning_curve(XGB_best, "XGBoost learning curves", X_train, Y_train, cv=kfold)
g = plot_learning_curve(LGB_best, "LGBM learning curves", x_train, y_train, cv=2)

### 4.6 Feature importance

In [ ]:
# name_models = {"RandomForest": RFC_best, "GradientBoosting": GBC_best, "XGBoost": XGB_best, "LightGBM": LGB_best}
name_models = {"LightGBM": LGB_best}
indices_num = 7

df_indices = pd.DataFrame(np.zeros((indices_num, len(name_models))), columns=name_models.keys())

for key in name_models:
    plt.figure()

    classifier = name_models[key]
    indices = np.argsort(classifier.feature_importances_)[::-1][:indices_num]

    df_indices[key] = X_train.columns[indices][:indices_num]

    g = sns.barplot(x=classifier.feature_importances_[indices][:indices_num], y=X_train.columns[indices][:indices_num], orient="h")

    g.set_xlabel("Relative importance", fontsize=12)
    g.set_ylabel("Features", fontsize=12)
    g.tick_params(labelsize=9)
    g.set_title(key + " feature importance")

df_indices

## 4. Prediction

In [ ]:
num_sold = pd.Series(LGB_best.predict(test), name='num_sold')
submission_data = pd.concat([test_id, num_sold], axis=1)
submission_data

In [ ]:
pred_y_val = pd.Series(LGB_best.predict(x_val), name='num_sold_pred')

In [ ]:
y_val_re = y_val.reset_index()

validation_data = pd.concat([y_val_re, pred_y_val], axis=1)

validation_data


In [ ]:
calc_smape_score(y_val, LGB_best.predict(x_val))

In [ ]:
#g = sns.relplot(x='index', y='num_sold', data=validation_data[:100], kind='line', height=20)
#g = sns.relplot(x='index', y='num_sold_pred', data=validation_data[:100], kind='line', height=20)

start=1000
end=start+50

fig, ax = plt.subplots(figsize=(20, 7))
sns.lineplot(validation_data[start:end]['index'], validation_data[start:end]['num_sold'], marker = 'o')
sns.lineplot(validation_data[start:end]['index'], validation_data[start:end]['num_sold_pred'], marker = 'x')


## 5. save submission data

In [ ]:
os.makedirs("../data/submission/", exist_ok=True)
submission_data.to_csv("../data/submission/20220120_LGBM.csv", index=False)

## Submit memo
- 20220120(1st try)
    - cross_validate, grid_search
    - best score in training
    ```
    Best Score: -6.462599703294936
    Best Param: {'feature_fraction': 1, 'learning_rate': 0.05, 'max_depth': 15, 'num_leaves': 31}
    ```
    - submittion score in kaggle
    ```
    6.81593 (ranking: 522/1099)
    ```
    - 所感
        - 結果の出方としては大体合ってるかな。
            - 評価指標の計算方法としては、問題なさそう。
        - feature importanceをみると、月日を重要視している。
            - これがいいのか悪いのか、微妙なところ
            - 年ごとの売り上げの違いとか、見た方がいいのかな
            - 月日がなかった場合の結果は、かなり変わるのかな

### 3.2 Hyperparameter tunning for best models

In [ ]:
for i in range(N_trials):
    print(f"Trial {i+1}")
    random_state = random.randint(0, 1000)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)

    for reg_name, reg in reg_dict.items():
        reg.fit(x_train,y_train)
        y_pred = reg.predict(x_test)
        mape = mean_absolute_percentage_error(y_test, y_pred)  # MAPEを算出
        mape_dict[reg_name].append(mape)  # 格納

In [ ]:
LGB_best.evals_result()

### 3.5 Ensemble modeling

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('gbc', GBC_best), ('xgb', XGB_best), ('lgb', LGB_best)], voting='soft', n_jobs=-1)

votingC = votingC.fit(X_train, Y_train)

In [ ]:
test_Survived = pd.Series(votingC.predict(test), name='Survived')

submission_data = pd.concat([IDtest, test_Survived], axis=1)

submission_data

## 6. check difference (extra)

In [ ]:
df_20220110 = pd.read_csv("../data/submission/20220110_ensemble_RF_GB.csv", index_col=0)
df_20220113 = pd.read_csv("../data/submission/20220113_ensemble_RF_GB_XGB_LGBM.csv", index_col=0)

In [ ]:
df_20220110[(df_20220110 == df_20220113).all(axis=1) == False]